In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import datetime as dt


import warnings
warnings.filterwarnings("ignore")

### 1. Load Data

In [2]:
pwd

"/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/Bella's Code"

In [54]:
test_result = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Pricing_Test/test_results.csv')
users = pd.read_csv('/Users/ChunyanHao/Desktop/GitHub/DS_Take_Home/data/Pricing_Test/user_table.csv')

In [8]:
test_result.head(3)

,user_id,timestamp,source,device,operative_system,test,price,converted
0,604839,2015-05-08 03:38:34,ads_facebook,mobile,iOS,0,39,0
1,624057,2015-05-10 21:08:46,seo-google,mobile,android,0,39,0
2,317970,2015-04-04 15:01:23,ads-bing,mobile,android,0,39,0


In [10]:
users.head(3)

,user_id,city,country,lat,long
0,510335,Peabody,USA,42.53,-70.97
1,89568,Reno,USA,39.54,-119.82
2,434134,Rialto,USA,34.11,-117.39


In [11]:
test_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316800 entries, 0 to 316799
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           316800 non-null  int64 
 1   timestamp         316800 non-null  object
 2   source            316800 non-null  object
 3   device            316800 non-null  object
 4   operative_system  316800 non-null  object
 5   test              316800 non-null  int64 
 6   price             316800 non-null  int64 
 7   converted         316800 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 19.3+ MB


In [12]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275616 entries, 0 to 275615
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  275616 non-null  int64  
 1   city     275616 non-null  object 
 2   country  275616 non-null  object 
 3   lat      275616 non-null  float64
 4   long     275616 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 10.5+ MB


In [13]:
for column in test_result.columns:
    uniques = test_result[column].unique()
    print('{0:20s} {1:5d}\t'.format(column, len(uniques)), uniques[:5])

user_id              316800	 [604839 624057 317970 685636 820854]
timestamp            140931	 ['2015-05-08 03:38:34' '2015-05-10 21:08:46' '2015-04-04 15:01:23'
 '2015-05-07 07:26:01' '2015-05-24 11:04:40']
source                  12	 ['ads_facebook' 'seo-google' 'ads-bing' 'direct_traffic' 'ads-google']
device                   2	 ['mobile' 'web']
operative_system         6	 ['iOS' 'android' 'mac' 'windows' 'other']
test                     2	 [0 1]
price                    2	 [39 59]
converted                2	 [0 1]


In [14]:
for column in users.columns:
    uniques = users[column].unique()
    print('{0:20s} {1:5d}\t'.format(column, len(uniques)), uniques[:5])

user_id              275616	 [510335  89568 434134 289769 939586]
city                   923	 ['Peabody' 'Reno' 'Rialto' 'Carson City' 'Chicago']
country                  1	 ['USA']
lat                    713	 [42.53 39.54 34.11 39.15 41.84]
long                   830	 [ -70.97 -119.82 -117.39 -119.74  -87.68]


In [55]:
test_data = test_result.merge(users, on = 'user_id', how = 'left')

In [56]:
test_data.head(2)

,user_id,timestamp,source,device,operative_system,test,price,converted,city,country,lat,long
0,604839,2015-05-08 03:38:34,ads_facebook,mobile,iOS,0,39,0,Buffalo,USA,42.89,-78.86
1,624057,2015-05-10 21:08:46,seo-google,mobile,android,0,39,0,Lakeville,USA,44.68,-93.24


In [57]:
test_data[['test', 'price', 'converted', 'user_id']].groupby(['test', 'price', 'converted'])['user_id'].count().reset_index()

,test,price,converted,user_id
0,0,39,0,198487
1,0,39,1,4030
2,0,59,0,205
3,0,59,1,5
4,1,39,0,154
5,1,39,1,1
6,1,59,0,112146
7,1,59,1,1772


##### We notice that there are a small pert of records in control group by with the price of 59, also some records in test group but with the price of 39.
##### Therefore we need to first drop these abnormal records.

In [74]:
test_data = test_data[(test_data['test'] == 0) & (test_data['price'] == 39)|
                     (test_data['test'] == 1) & (test_data['price'] == 59)] 

In [75]:
convert_temp = test_data[['test', 'converted', 'price']].groupby(['test', 'price'])['converted'].mean().reset_index()
convert_temp.columns = ['control or test', 'conversion_rate']


count_temp = test_data[['test', 'converted', 'user_id']].groupby(['test', 'converted'])['user_id'].count().reset_index()
count_temp.columns = ['control or test', 'converted', 'user_count']

print(convert_temp)
print()
print(count_temp)

   control or test  conversion_rate
0                0         0.019900
1                1         0.015555

   control or test  converted  user_count
0                0          0      198487
1                0          1        4030
2                1          0      112146
3                1          1        1772


### 2. Data Visualization

##### From the table above we notic that the conversion_rate in test group is 0.44% lower than the control group
##### Howerver, before we get the final conclusion, we need to confirm that the sample distribution between the control and test group are random.
##### In other words, the only difference between test and control group is price